In [38]:
!pip install pyspark

In [39]:
import os
import sys
import re
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [40]:
# criar e inicializar a spark session
spark = SparkSession.builder.appName("AIR_CIA").getOrCreate()

In [42]:

#leitura dos CSVs utilizando UTF-8 como encoding
df_air_cia = spark.read.option("delimiter", ";").option("encoding", "UTF-8").csv('/content/AIR_CIA/', header=True)

#verificacao de schema pois csv infere tudo como 'string'
df_air_cia.printSchema()


root
 |-- Razão Social: string (nullable = true)
 |-- ICAO IATA: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- Atividades Aéreas: string (nullable = true)
 |-- Endereço Sede: string (nullable = true)
 |-- Telefone: string (nullable = true)
 |-- E-Mail: string (nullable = true)
 |-- Decisão Operacional: string (nullable = true)
 |-- Data Decisão Operacional: string (nullable = true)
 |-- Validade Operacional: string (nullable = true)



# passando para snake case as colunas:

In [43]:
# conversao para snake case
for column in df_air_cia.columns:
    new_column_name = column.lower().replace(" ", "_").replace("-", "_")
    df_air_cia = df_air_cia.withColumnRenamed(column, new_column_name)

In [44]:
df_air_cia.show(7)

+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|        razão_social|icao_iata|              cnpj|   atividades_aéreas|       endereço_sede|            telefone|              e_mail| decisão_operacional|data_decisão_operacional|validade_operacional|
+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|ABSA - AEROLINHAS...|   LTG M3|00.074.635/0001-33|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              22/04/2015|          23/04/2025|
|AEROSUL TÁXI AÉRE...|   ASO 2S|27.315.694/0001-02|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO Nº 282|              10/02/2021|       

In [45]:
# retirar os caracteres especiais das colunas
for col_name in df_air_cia.columns:
    new_col_name = col_name \
        .replace("ã", "a") \
        .replace("á", "a") \
        .replace("à", "a") \
        .replace("é", "e") \
        .replace("ê", "e") \
        .replace("í", "i") \
        .replace("ó", "o") \
        .replace("ô", "o") \
        .replace("õ", "o") \
        .replace("ú", "u") \
        .replace("ç", "c") \
        .replace("-", "_")

    df_air_cia = df_air_cia.withColumnRenamed(col_name, new_col_name)

In [46]:
df_air_cia.show(truncate=False)

+------------------------------------------------------------------+---------+------------------+------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+-----------------------------------+----------------------------+------------------------+--------------------+
|razao_social                                                      |icao_iata|cnpj              |atividades_aereas                                                                               |endereco_sede                                                                                                                                                       |telefone                                     |e_mail                             |decisao_operacional         |data_decisao_

# Operacao de slice da coluna 'icao_iata' para 'icao' e 'iata':

In [47]:
# Operacao de slice da coluna 'icao_iata' para 'icao' e 'iata'
df_air_cia = df_air_cia.withColumn("icao_iata", split("icao_iata", " "))
df_air_cia = df_air_cia.withColumn("icao", df_air_cia["icao_iata"].getItem(0))
df_air_cia = df_air_cia.withColumn("iata", when(df_air_cia["icao_iata"].getItem(1).isNotNull(), df_air_cia["icao_iata"].getItem(1)))

In [48]:
# trazendo as colunas 'icao' e 'iata' obedecendo a ordem do dataframe original
df_air_cia = df_air_cia.select("razao_social", "icao", "iata", "cnpj", "atividades_aereas", "endereco_sede", "telefone", "e_mail", "decisao_operacional", "data_decisao_operacional", "validade_operacional")
# substitui os valores nulos por uma string vazia
df_air_cia = df_air_cia.na.fill(' ')
# preview final do data frame antes de exporta-lo
df_air_cia.show(truncate=False)

+------------------------------------------------------------------+----+----+------------------+------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+-----------------------------------+----------------------------+------------------------+--------------------+
|razao_social                                                      |icao|iata|cnpj              |atividades_aereas                                                                               |endereco_sede                                                                                                                                                       |telefone                                     |e_mail                             |decisao_operacional         |data_decisao_

In [49]:
# como nesse caso o arquivo é pequeno vamos compactar em um, para salva-lo
df_air_cia = df_air_cia.coalesce(1)

In [51]:
# exportacao do arquivo assegurando 'encoding = UTF-8'
df_air_cia.write.option("encoding", "UTF-8").parquet("/content/AIR_CIA.parquet", mode="overwrite")